# 1. Übungsblatt

**Hinweis:** <br>
Wir importieren das Package `jdc` um die Definition dieser Klasse auf mehrere Zellen zu verteilen und somit zwischendurch Hilfestellung geben zu können. Wir werden dieses Package auch in zukünftigen Übungsblättern verwenden. Sie müssen sich jedoch nicht weiter mit diesem Package auseinandersetzen, da alles notwendige vorgegeben sein wird. Für alle Interessierten:
* wir benötigen `!pip install jdc` um das Package zu installieren, da das Packages standardmäßig nicht installiert ist.
* mit `import jdc` importieren wir das Package
* wir schreiben `%%add_to CLASSNAME` in die erste Zeile einer Zelle (oder alternativ nach den Package imports) um alle Methoden dieser Zelle der Klasse `CLASSNAME` zuzuordnen.


In [74]:
# Um Klassendefinition auf mehrere Zellen aufteilen zu können.
import jdc
import numpy as np
import copy
import heapq

## Aufgabe 1.4 (A*-Suche)
In dieser Aufgabe soll die A$^\star$-Suche implementiert werden. Hierfür definieren wir als erstes die Klasse `Node`. Sie können diese auch nach ihren eignen Vorstellungen ändern. Wir finden folgende Attribute sinnvoll:
  * Name: Name des Knotens
  * child_weight_list: Eine Liste von Tupeln $[(v_1, n_1), \ldots, (v_k, n_k)]$, wobei $v_i$ der Namen eines Nachbarknotens und $n_i$ die Kosten zwischen dem aktuellen Knoten und $v_i$ entsprechen.
  * predecessor: Speichert den Vorgängerknoten
  * g: Die bisherigen Kosten zwischen dem Startknoten und diesem Knoten
  * h: Eie heuristischen Kosten
  * f: Die Gesamtkosten.

Des Weiteren soll eine Methode `set_children(self, child_weight_list)` geschrieben werden um nachträglich noch Kinder bzw Nachbarknoten zu setzen.

Damit wir später ein `Node`-Objekt in die Queue legen können, müssen wir festlegen, wie zwei `Node`-Objekte verglichen werden können. Hierfür implementieren wir die \_\_lt\_\_() Methode (lt steht für "less than"). Zur Erinnerung, ein Knoten $u$ wird vor einem anderen Knoten $v$ besucht, wenn $$f(u) < f(v).$$
Genau das implementieren wir in der Methode `__lt__(self, other)`. Diese Methode müssen Sie nicht mehr ändern.

In [75]:
class Node(object):
    def __init__(self, name, neighbor_weight_list, h):
        self.name = name
        self.neighbor_weight_list = neighbor_weight_list
        self.predecessor = None
        self.h = h
        self.g = 0 # 10**3
    
    def f(self):
        return self.h + self.g
        
    def set_neighbors(self, neighbor_weight_list):
        self.neighbor_weight_list = neighbor_weight_list   
        
    def __lt__(self, other):
        return self.f() < other.f()
    
    def __str__(self):
        return self.name

Definieren Sie jetzt den Graphen aus Aufgabe 1.2. a und b.

In [76]:
def graph_1(aufgabenteil_a):
    S = Node(name="S", neighbor_weight_list=[], h=128)
    A = Node(name="A", neighbor_weight_list=[], h=75)
    B = Node(name="B", neighbor_weight_list=[], h=104)
    C = Node(name="C", neighbor_weight_list=[], h=139)
    D = Node(name="D", neighbor_weight_list=[], h=160)
    E = Node(name="E", neighbor_weight_list=[], h=90)
    F = Node(name="F", neighbor_weight_list=[], h=37)
    if aufgabenteil_a:
        G = Node(name="G", neighbor_weight_list=[], h=27)
    else:
        G = Node(name="G", neighbor_weight_list=[], h=3)
    T = Node(name="T", neighbor_weight_list=[], h=0)

    S.set_neighbors(neighbor_weight_list=[(B, 29), (C, 12), (A, 53)])
    A.set_neighbors(neighbor_weight_list=[(S, 53), (E, 9)])
    B.set_neighbors(neighbor_weight_list=[(S, 29), (D, 67), (E, 32)])
    C.set_neighbors(neighbor_weight_list=[(S, 12), (D, 24)])
    D.set_neighbors(neighbor_weight_list=[(B, 67), (C, 24)])
    E.set_neighbors(neighbor_weight_list=[(A, 9), (B, 32), (F, 58), (G, 73)])
    F.set_neighbors(neighbor_weight_list=[(E, 58), (G, 14), (T, 40)])

    if aufgabenteil_a:
        G.set_neighbors(neighbor_weight_list=[(E, 73), (F, 14), (T, 28)])
        T.set_neighbors(neighbor_weight_list=[(F, 40), (G, 28)])
    else:
        G.set_neighbors(neighbor_weight_list=[(E, 73), (F, 14), (T, 23)])
        T.set_neighbors(neighbor_weight_list=[(F, 40), (G, 23)])

    return S


def graph_2():
    s = Node(name="s", neighbor_weight_list=[], h=40)
    b = Node(name="b", neighbor_weight_list=[], h=30)
    c = Node(name="c", neighbor_weight_list=[], h=0)
    a = Node(name="a", neighbor_weight_list=[], h=0)
    t = Node(name="t", neighbor_weight_list=[], h=0)

    s.set_neighbors(neighbor_weight_list=[(b, 10), (a, 25)])
    b.set_neighbors(neighbor_weight_list=[(s, 10), (c, 20)])
    c.set_neighbors(neighbor_weight_list=[(b, 20), (a, 10), (t, 10)])
    a.set_neighbors(neighbor_weight_list=[(s, 25), (c, 10)])
    t.set_neighbors(neighbor_weight_list=[(a, 10)])

    return s

Implementieren Sie jetzt die Methode `a_star(self, target_node_name)` so, dass eine A$^\star$-Suche ausgehend vom aktuellen Knoten `self` zum Zielknoten mit dem Namen `target_node_name` durchgeführt wird und dabei den kürzesten Weg findet, auch wenn die Heuristik nicht monoton ist (d.h. ohne closed_list). In der nächsten Zelle sehen Sie, wie ein `Node`-Objekt in eine Priority-Queue gelegt werden kann. Weitere Informationen zur Verwendung des queue Packages finden Sie auf dem 0. Übungsblatt.

In [77]:
%%add_to Node

def a_star(self, target_node_name):
    visited = []
    found = [self]
    heapq.heapify(found)

    while(found):
        current = heapq.heappop(found)
        visited += [current]
        if current.name == target_node_name:
            path = []
            while (current.predecessor):
                path = [current.name] + path
                current = current.predecessor
            return [self.name] + path
        for (n, d) in current.neighbor_weight_list:
            if n in found and n.g < current.g + d:
                continue
            if n in visited and n.g < current.g + d:
                continue
            n.g = current.g + d
            n.predecessor = current
            if not n in found:
                heapq.heappush(found, n)

Wenden Sie die A* Suche so an, wie in Aufgabe 1.2 a und b beschrieben. Lassen Sie sich zum Schluss den kürzesten (S,T)-Pfad ausgeben.

In [78]:
print(graph_1(True).a_star("T"))

['S', 'B', 'E', 'F', 'T']


## Bonusaufgabe (Alpha-Beta Pruning)
In dieser Aufgabe soll die  MinMax Suche mit Alpha-Beta Pruning implementiert werden. Dazu definieren wir unsere Klasse `Node` neu, denn Attibute wie `h`, `g`, und `f` benötigen wir jetzt nicht mehr. Stattdessen können folgende Attribute nützlich sein:
  * value: Der Wert des Knotens.
  * id: Ein eindeutiger Wert für den Knoten. Da es mehrere Knoten mit dem gleichen Wert geben kann, ist es sinnvoll eine zusätzliche Zahl oder String zu verwenden um den Knoten eindeutig identifizierbar zu machen.
  * is_maximizingPlayer: Ein Status-Flag das speichert, ob es sich bei dem Knoten um einen Min- oder Max-Spieler handelt.
  * alpha: Der alpha-Wert.
  * beta: Der beta-Wert.
  * children: Die Kinder der aktuellen Knotens.


mit dem zugehörigen Konstruktor. Das Grundgerüst dazu ist bereits in der nächsten Zelle angegeben und kennen Sie vielleicht schon aus Übungsblatt 0. Sie können den Konstruktor um zusätzliche Parameter (z.B. `alpha`, `beta` und `value`) erweitern.

In [79]:
class Node:
    def __init__(self, id, is_maximizingPlayer, children, alpha=None, beta=None, value=None,):
        self.id = id
        self.value = value
        self.is_maximizingPlayer = is_maximizingPlayer

        self.alpha = alpha
        self.beta = beta

        self.children = children

Verwenden Sie jetzt diese Klasse um den Suchbaum aus Aufgabe 1.2 zu konstruieren. Wenn Sie möchten, können Sie diese Konstruktion in eine Funktion packen um eine bessere Übersicht zu gewähren und die Sichtbarkeit der Variablennamen einzuschränken.

In [80]:
def tree_1():
    leaf_7 = Node(id=11, value=7, is_maximizingPlayer=False, children=[])
    leaf_15 = Node(id=12, value=15, is_maximizingPlayer=False, children=[])
    leaf_20 = Node(id=13, value=20, is_maximizingPlayer=False, children=[])
    leaf_25 = Node(id=14, value=25, is_maximizingPlayer=False, children=[])
    leaf_16 = Node(id=15, value=16, is_maximizingPlayer=False, children=[])
    leaf_3_left = Node(id=16, value=3, is_maximizingPlayer=False, children=[])
    leaf_21 = Node(id=17, value=21, is_maximizingPlayer=False, children=[])
    leaf_6 = Node(id=18, value=6, is_maximizingPlayer=False, children=[])
    leaf_17 = Node(id=19, value=17, is_maximizingPlayer=False, children=[])
    leaf_10 = Node(id=20, value=10, is_maximizingPlayer=False, children=[])
    leaf_3_right = Node(id=21, value=3, is_maximizingPlayer=False, children=[])
    leaf_12 = Node(id=22, value=12, is_maximizingPlayer=False, children=[])
    leaf_8 = Node(id=23, value=8, is_maximizingPlayer=False, children=[])
    leaf_13 = Node(id=24, value=13, is_maximizingPlayer=False, children=[])
    leaf_2 = Node(id=25, value=2, is_maximizingPlayer=False, children=[])
    leaf_11 = Node(id=26, value=11, is_maximizingPlayer=False, children=[])

    inner_node_1 = Node(id=4, is_maximizingPlayer=True, children=[leaf_7, leaf_15])
    inner_node_2 = Node(id=5, is_maximizingPlayer=True, children=[leaf_20, leaf_25])
    inner_node_3 = Node(id=6, is_maximizingPlayer=True, children=[leaf_16, leaf_3_left, leaf_21])
    inner_node_4 = Node(id=7, is_maximizingPlayer=True, children=[leaf_6, leaf_17, leaf_10])
    inner_node_5 = Node(id=8, is_maximizingPlayer=True, children=[leaf_3_right, leaf_12])
    inner_node_6 = Node(id=9, is_maximizingPlayer=True, children=[leaf_8, leaf_13])
    inner_node_7 = Node(id=10, is_maximizingPlayer=True, children=[leaf_2, leaf_11])

    inner_node_8 = Node(id=2, is_maximizingPlayer=False, children=[inner_node_1, inner_node_2, inner_node_3])
    inner_node_9 = Node(id=3, is_maximizingPlayer=False, children=[inner_node_4, inner_node_5, inner_node_6, inner_node_7])

    root = Node(id=1, is_maximizingPlayer=True, children=[inner_node_8, inner_node_9], alpha=-np.inf, beta=np.inf)

    return root

## Aufgabe 1.3 (Alpha-Beta-Pruning)
In dieser Aufgabe soll die  MinMax Suche mit Alpha-Beta Pruning implementiert werden. Zunächst definieren wir eine Klasse `Node` mit dem zugehörigen Konstruktor. Das Grundgerüst dazu ist bereits in der nächsten Zelle angegeben und kennen Sie vielleicht schon aus Übungsblatt 0. Sie können den Konstruktor um zusätzliche Parameter (z.B. `alpha`, `beta` und `value`) erweitern.
<br>
<br>
Jetzt soll die Methode `alpha_beta(...)` für die Klasse `Node` implementiert werden. Jenachdem welche Parameter Sie dem Konstruktor übergeben, benötigt die Methode `alpha_beta` mehr oder weniger Parameter. 
<br>
Hinweis:
Es gibt eine Möglichkeit, sodass sie der Methode `alpha_beta` gar keine Parameter übergeben müssen.

In [81]:
%%add_to Node
def alpha_beta(self, alpha, beta):
    if(not self.children):
        return self.value
    
    if (self.is_maximizingPlayer):
        maxEva = -np.inf

        for c in self.children: 
            maxEva= max(maxEva, c.alpha_beta(alpha, beta))
            alpha = max(alpha, maxEva)
            if (beta <= alpha):
              break
        return maxEva

    else:
        minEva = np.inf

        for c in self.children: 
            minEva= min(minEva, c.alpha_beta(alpha, beta))
            beta = min(beta, minEva)
            if (beta <= alpha):
                break
        return minEva

Als nächstes rufen wir die Methode auf dem Wurzelknoten auf.

In [82]:
tree_1().alpha_beta(-np.inf, np.inf)

15